# Determine Costas Array Frequencies, Order 8, For Yaesu Radios
## By Terry Bondy, VA3TYB

In [1]:
source("../common/preamble.m")

Last updated: Sunday 16 February 2020

In [2]:
source("../common/radio.m")

radioFreqAudioBwLow =  200
radioFreqAudioBwHigh =  2600
radioFreqAudioBw =  2400
radioFreqAudioMidBand =  721.11


Want to spread the frequencies equidistant across the audio bandpass for the radio, but have it centered one the geometric center of the audio bandpass for the radio. That means:

 1. $N = 8$,
 1. $costas8Freq_{8} = costas8Freq_{1} + costas8FreqDelta * (N - 1)$
 1. $\frac{costas8Freq_{1}}{radioFreqAudioBwLow} = \frac{radioFreqAudioBwHigh}{costas8Freq_{8}}$
 1. $costas8Freq_{8} < radioFreqAudioBwHigh$
 1. $radioFreqAudioBwLow < costas8Freq_{1}$
 
Using 1, 2, 4, and 5 an upper limit can be set on $costas8FreqDelta$.
$costas8FreqDelta < \frac{radioFreqAudioBwHigh - radioFreqAudioBwLow}{7}$
or 

In [3]:
(radioFreqAudioBwHigh - radioFreqAudioBwLow)/7

ans =  342.86


So let set

In [4]:
costas8FreqDelta = 300

costas8FreqDelta =  300


Substituting into 3. and re-arranging

 * $costas8Freq_{1} \cdot costas8Freq_{8} = radioFreqAudioBwLow \cdot radioFreqAudioBwHigh$
 * $costas8Freq_{1} \cdot (costas8Freq_{1} + 300 \cdot 7) = 200 \cdot 2600$
 * $costas8Freq_{1}^{2} + costas8Freq_{1} \cdot 2100 = 520000$
 
Solving for the positive root provides

In [5]:
roots([1, 2100, -520000])

ans =

  -2323.77
    223.77



Checking

In [6]:
(2323.77-223.77)/7

ans =  300


In [7]:
223.77/radioFreqAudioBwLow

ans =  1.1189


In [8]:
radioFreqAudioBwHigh/2323.77

ans =  1.1189


Setting to a reasonable nearby integer

In [9]:
costas8FreqsCv = [225:costas8FreqDelta:2325](:)

costas8FreqsCv =

    225
    525
    825
   1125
   1425
   1725
   2025
   2325



Need a frequency ordering. From _Modified Costas Signal, Levanon & Mozeson_ here is one:

In [10]:
costas8FreqIdxCv = [ 1 8 3 6 2 7 5 4 ](:);